# RAG (langchain 밋업 발표 정리)

## RAG
Retrieval Augmented Generation

**RAG 프로세스** : 전처리 작업 > 서비스 단계에서 이뤄지는 작업
- 전처리 작업 : Load, split(청킹 - 나누는 작업),EMBED (벡터 공간에 저장), STORE(벡터 데이터 베이스에 저장)
- 서비스 단계에서 이뤄지는 작업 : LLM 서비스가 이뤄지면서 유저의 질문을 RETRIEVE 해서 문서를 발췌해서 원하는 답변 출력

### 문서 전처리 단계
1. Document Loader : 문서 로드 (150)
2. Text Splitter : 분할 전략 (10)
3. Embedding : 임베딩 (60+huggingface)
4. Vector Store : Vector DB (80)
5. Retrievers : Vector DB 검색기 (50)


상황에 맞는 적절한 조합을 선택해야한다.



### Document Loader(PDF)
- 취급할 데이터는 주로 csv, Excel, PDF 형식
- 일간 Market 지를 유료 구독하여 시장 분석에 활용하는 경우도 많음
- Market지 뿐만 아니라 회사의 주요 문서도 보안에 유리한 PDF 형식의 문서가 많음

### Document Loader 선택시 고려사항
- 텍스트를 원형 그대로 가져오는가? (한글 인코딩, 특수문자)
- 메타데이터의 종류는 어떤 것들이 있는가? (page_context:문서내용, 페이지번호, 표, 차트, 문서의 좌표, 속성 등)
- 문서를 읽는 속도는 얼마나 빠른가? (업로드 시 문서를 읽는 "속도"가 중요함)

### PDF Loader
**fitz**
- 단순하게 모든 Text를 읽어서 하나의 문자열을 합칠 때 유용
- 페이지를 읽는 속도가 가장 빠름
- 페이지 번호 제공
- 페이지 번호에 제외한 metadata 미지원

**pyPDFLoader**
- 예제 코드에 가장 많이 출현
- 평균적으로 우수함
- page 단위로 데이터를 로드
- metadata(source:파일명, page)

**UnstructurePDFLoader**
- 가장 많은 metadata 정보 제공
- 요소별 Load 가능(좌표 등)
- 단, 속도가 다른 Loader 대비 느림

**PDFPlumber**
- 한글 인코딩 처리 능력이 우수, 다양한 metadata 정보 제공
- 다양한 metadata 정보를 포함(작성자, ModDate)
- 읽기 속도가 가장 느림



### Text Splitter 
- 문서를 특정 기준으로 분할(Chunk) 할 때 활용


**Character TextSplitter**
- 분할 가능한 최소의 단위로 분할을 시도(공백이나 "."을 기준으로 분할)
- 중요한 문서의 소제목에서 텍스트가 잘릴 수 있음 (**chunk_overlap이 궁극적인 해결책이 안되는 경우가 많음**)


**RecursiveCharacterTextSplitter**
- 범용적으로 많이 사용되는 분할 방식
- 청크가 충분히 작아질 때까지 순서대로 분할하려고 시도
- 단락 => 문장 => 단어 순소로 함께 유지하려고 시도하는 효과가 있는데, 이는 일반적으로 텍스트의 가장 강력한 의미 관련 부분으로 보이기 때문


**TokenTextSplitter**
- 토큰 단위로 분할
- TokenTextSplitter를 바로 사용하면 한글 처리가 모호함
- 따라서 konlpyTextSplitter를 사용하면 해결책이 될 수 있음
- kkma 은 분석적 심도가 우선시되는 애플리케이션에 적합


**오픈소스**
-HuggingFace에 업로드 된 다양한 토크나이저를 쉽게 활용 가능
- 특정 도메인 용어(의료 용어), 신조어 등 Fine-Tuning 혹은 단순 추가 가능


**SemanticChunker**
- langchain_experimental에 신규 추가된 Chunker
- 텍스트를 의미 유사성에 따라 분할
- 다른 토크나이저와 달리 chunk_size, chunk_overlap과 같은 파라미터를 initialize에 사용하지 않는 것이 특징


### Embedding
- 임베딩은 텍스트의 벡터 표현을 만듬.
- 벡터 공간에서 가장 유사한 텍스트를 찾는 Semantic Search(의미검색)과 같은 작업을 수행할 수 있기 때문에 유용
- 따라서, 문서에 적합한 Embedding 모델을 선택하는 것은 중요함 (한국어)까지 고려
- LangChain의 기본 Embeddings 클래스는 두 가지 메서드를 제공함
    1. Document(문서) Embedding
    2. Query Embedding
- 주요 Embedding 목록
    - OpenAIEmbedding
    - OpenSource(HuggingFace) > BGE, Mistral



**OpenAIEmbedding**
- OpenAI 에서 제공하는 유료 embedding
- 토큰 당 과금이 되는 방식
- 사용방식이 편리하면서 성능이 보장된다는 이점이 있음
- 하지만 문서의 양이 많다면 비용이 너무 많이 발생함


**ChasBackedEmbeddings**
- 임베딩을 저장하거나 임시로 캐싱하여 다시 계산할 필요가 없도록 함 
- 임베딩을 key-value 저장소에 캐싱
- 텍스트는 해시되고 해시는 캐시에서 키로 사용
- 여러 embedding 모델을 사용하는 경우 namespace를 별도로 설정하여, 1개의 문서에 대한 서로 다른 캐싱 가능
- 캐싱 Embedding 벡터를 가져오는 것은 매우 빠르게 조회가 가능
(과금 방지를 위해서 랩핑해서 꼭 사용하기)

**OpenAIEmbedding + CacheBackedEmbeddings**


**MTEB(Massive Text Embedding Benchmark)**
- Average : 67.56


**OpenSource Embeddings - HuggingFace**
- 공개되어 있는 Embedding 모델도 훌륭한 성능을 보여주고 있음
- 하지만 한글에 대한 성능을 고려해야 함

#### 정리
##### OpenSource
- 문서가 주로 영문으로 되어 있다면 공개되어 있는 모델을 활용하는 것이 비용적인 측면과 향후 유지 보수에 유리함
- Embedding 모델을 구동할 수 있는 서버가 뒷받침 되어야 함
- HuggingFace Inference API 활용을 고려해볼 수 있음

##### OpenAIEmbedding
- 서버 인프라 / RAG pipeline 인프라 구축이 힘든 상황이라면, 선택
- 다국어(한국어) 문서가 주를 이루는 경우라면 안정적인 선택지
- Cache를 반드시 적용하여 불필요한 과금을 줄여야 함

### VectorStore
- Langchain은 수십개의 VectorStore와의 연동성을 제공함

**Vector DB의 중요성**
- 대규모 생성 AI 모델에 대한 필요성이 증가함에 따라 벡터 데이터베이스의 중요성이 부각

**배경**
    - AI 모델이 복잡하고 방대한 데이터를 필요로 하면서 데이터 폭증이 일어남
    이러한 상황에서 벡터 데이터베이스는 데이터 유입을 효율적으로 관리하는데 중요
    - AI 모델이 생성하는 복잡한 텍스트는 고급 유사성 검색 및 매칭을 요구함
    여기서 전통적인 검색보다 더 뛰어난 관련성과 정확도를 제공

- VectorStore 마다 고유 특성이 있으므로, 여러개의 특성을 이해하고 적응해야 함


**Semantic Search**
- 벡터 데이터베이스는 각 회사별 고유의 알고리즘으로 search 기능을 제공하며, 알고리즘에 따라 검색 결과가 상이함
- 인기 있는 Vector DB (Pinecone, Weaviate, ElasticSearch, Chroma, FAISS)


**FAISS**
- Dense Vector 의 유사성을 신속하게 검색하고 클러스터링하는 오픈소스 라이브러리
- RAM 용량을 초과하는 다양한 크기의 벡터 세트 내 검색할 수 있는 알고리즘을 포함

## 전처리 Tip

페이지 단위 분할
- 전체 문서를 페이지 단위로 분할
- 중요한 metadata 정보 태깅 (페이지 번호, 파일명, MOD-Modified data, AUTHOR, 키워드 추출)


필요한 영역 Crop
- 여백영역에 표기된 정보가 추후 검색시 노출
- 따라서, main 영역 crop하여 불필요한 정보 제거


1:1 분할 문서 Parsing
- 열간 분리된 상태에서 해야함
- Bounding Box 계산 후 텍스트를 순서대로 읽는 방식 (PDFPlumber)
- PDFminer를 사용해서 자동으로 분할된 컬럼에 대한 처리
**사전에 차트나 표를 제거 후 작업, 한 줄에 글자수 <N 개 이하는 제거**


표 추출(1)
- 중요한 정보가 많고 표 정보를 정형화된 데이터로 추출하는 것이 중요
- camelot, PaddleOCR 라이브러리 활용


표 추출(2)
- metadata (파일명, 페이지 번호, 객체 번호)


Chunk Overlap은 넉넉하게
- 이전 Page 뒷부분 + 다음 Page 앞부분 병합 후 > Semantic Chunker 처리(단 페이지 번호 처리 모호)
- 한 문서에 대해서 여러 Chunk size를 적용 후 retrieval 결과의 Union을 최종결과로 return 


이미지 추출
- fitz를 사용하여 쉽게 이미지만 추출 가능
- 이미지 추출 및 태깅을 해두는 것이 중요

### Retriever

**Multi-Query Retriever(1)**
- 거리 기반 벡터 데이터베이스 검색은 유사한 Embedding된 문서를 찾음
- 그러나 Query 문구가 미묘하게 변경되거나 Embedding이 데이터의 의미를 제대로 포착하지 못하는 경우 검색 결과가 달라질 수 있음

- 방식
    - LLM을 사용해 주어진 사용자 입력 쿼리에 대해 서로 다른 관점에서 여러 쿼리를 생성함으로써 프롬프트 튜닝 프로세스를 자동화
    - 각 쿼리에 대해 관련 문서 집합을 검색하고 모든 쿼리에서 고유한 유니온을 사용하여 잠재적으로 관련성이 높은 더 큰 문서 집합을 가져옴 


**Multi-Query Retriever(2)**
- 질문에 대해서 여러개의 잠재적인 Query에 대해서 만들어 줌
- 그 결과에 대한 유니온 값을 반환


**Emsemble Retriever(1)**
- semantic search가 유사 의미를 가지는 문서의 검색에서는 유리하나, 특정 키워드가 반드시 포함되어야 하는 검색시 유사 단어가 포함된 문서가 검색될 수 있음


**Emsemble Retriever(2)**
- 1개 이상의 Retriever 를 앙상블하여 사용
    - Sparse Retriever + Dense Retriever 의 앙상블
- 서로 다른 알고리즘의 강점을 활용함으로써 앙상블 리트리버는 어떤 단일 알고리즘보다 더 나은 성능을 얻을 수 있음
- Retriever를 선택할 수 있음은 물론이고, 앙상블하는 Retriever간 weight 조절도 가능


**Long Context Reorder(1)**
- 모델의 아키텍처에 관계없이 검색된 문서를 10개 이상 포함하면 상당한 성능 저하 발생
- 모델이 긴 컨텍스트 중간에 관련 정보에 엑세스해야 하는 경우, 제공된 문서를 무시하는 경향


**Long Context Reorder(2)**
- 매우 중요 - 덜 중요 - 중요 순서로 정렬
- 따라서 중요한 문서 검색 결과를 상위로 재조정하여 중요한 정보를 답변에 포함하도록 유도


**그 밖에**
- Multi-Vector Retriever : 여러 VectorStore의 Retriever를 앙상블
- ContextualCompressor : 긴 길이의 문서 검색 결과를 압축
- LLMChainFilter : 문서의 검색 결과를 필터링

### Prompt Engineering


- 양식에 맞춘 출력 형식이 중요한 경우 (사업보고서, 기획서)
    - FewshotpromptTemplate 활용
- LangSmith Hub에 업로드 되어 있는 완성형 Prompt 을 pull하여 사용하기
- 별도의 .yaml 파일로 프롬프트 version 관리


**문서요약 프롬프트-Chain of Density(1)**
- 요약에 포함할 정보의 "적절한"양을 선택하는 것은 어려움
- 좋은 요약은 정보의 밀도가 높고 따라가기 어렵지 않으면서 자세하고 Entity 중심
- Chain of Density를 통해서 점점 더 밀도가 높은 요약을 요청
- 5번의 반복을 통해서 요약본을 적음
- Missing Entities를 추가 및 갱신하여 다음 요약 때 Denser Summary로 출력
